In [ ]:
import pandas as pd
import numpy as np
import torch
# Set seed for random
torch.manual_seed(68)
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math


In [ ]:
!pip install ray
!pip install optuna
!pip install -U tensorboardx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from functools import partial

In [ ]:
data = pd.read_csv('./data_post_covid.csv', index_col = 'Date')
data

,Open,High,Low,Close,Volume,DX-Y.NYB_Close,CL=F_Close,^VIX_Close,GC=F_Close,MSFT_Close,...,21middle,21lower,14rsv,21rsv,FT_3components,FT_6components,FT_9components,FT_27components,FT_81components,FT_100components
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-04,5.932000,6.016500,5.787000,5.797000,255680000,79.919998,73.139999,26.080000,1062.400024,20.917377,...,6.248762,5.763647,21.630612,17.794448,83.596852,105.502660,101.283516,111.174051,111.090952,111.275050
2010-02-05,5.794000,5.882500,5.705000,5.869500,220496000,80.440002,71.190002,26.110001,1052.199951,21.052624,...,6.213381,5.731172,20.281010,17.497085,83.398979,104.947948,100.395843,108.419804,101.622448,99.675144
2010-02-08,5.969000,6.050000,5.828000,5.841500,197804000,80.300003,71.889999,26.510000,1065.699951,20.827211,...,6.182024,5.692854,16.084298,14.423536,83.201042,104.392242,99.507848,105.656495,92.231059,88.195386
2010-02-09,5.910000,5.954500,5.850000,5.901500,184460000,79.860001,73.750000,26.000000,1076.699951,21.045107,...,6.145143,5.695302,19.948238,18.437731,83.003045,103.835569,98.619608,102.886140,82.977146,76.944542
2010-02-10,5.900000,5.930500,5.800000,5.868000,124664000,80.029999,74.519997,25.400000,1075.800049,21.030079,...,6.114309,5.681808,19.892770,18.990430,82.804988,103.277953,97.731200,100.110761,73.919182,66.027394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,226.940002,229.139999,226.130005,229.050003,15007500,108.260002,70.099998,14.270000,2620.000000,438.450836,...,220.836666,202.592754,64.735975,81.595876,84.585221,108.260431,105.714376,124.739798,157.430199,167.219500
2024-12-26,228.500000,228.500000,226.669998,227.050003,16146700,108.129997,69.620003,14.730000,2638.800049,437.233276,...,222.055714,205.848915,62.371983,82.471506,84.387685,107.711067,104.829461,122.060589,148.500439,156.662792
2024-12-27,225.600006,226.029999,220.899994,223.750000,27367100,108.000000,70.599998,15.950000,2617.199951,429.668457,...,222.812381,207.894857,56.107888,78.919673,84.190079,107.160582,103.943841,119.362466,139.341305,145.686180


In [ ]:
x_pca = StandardScaler().fit_transform(data)
pca = PCA(n_components=.95)
principalComponents = pca.fit_transform(x_pca)

print(f'PCA shape: {principalComponents.shape}')

#principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
data['y'] = data['Close']

x = data.iloc[:, :42].values
y = data.iloc[:, 42].values

#Split dataset into train (%70), test (%15) and val (%15)
train_x = x[:int(0.7*len(x))]
train_y = y[:int(0.7*len(y))]
test_x = x[int(0.7*len(x)):int(0.85*len(x))]
test_y = y[int(0.7*len(y)):int(0.85*len(y))]
val_x = x[int(0.85*len(x)):]
val_y = y[int(0.85*len(y)):]


print(f'trainX: {train_x.shape} trainY: {train_y.shape}')
print(f'testX: {test_x.shape} testY: {test_y.shape}')
print(f'valX: {val_x.shape} valY: {val_y.shape}')

x_scaler = MinMaxScaler(feature_range = (0, 1))
y_scaler = MinMaxScaler(feature_range = (0, 1))

train_x = x_scaler.fit_transform(train_x)
test_x = x_scaler.transform(test_x)
val_x_scaled = x_scaler.transform(val_x)

train_y = y_scaler.fit_transform(train_y.reshape(-1, 1))
test_y = y_scaler.transform(test_y.reshape(-1, 1))
val_y_scaled = y_scaler.transform(val_y.reshape(-1, 1))


trainX: (2626, 42) trainY: (2626,)
testX: (563, 42) testY: (563,)
valX: (563, 42) valY: (563,)


In [ ]:
def sliding_window(x, y, window):
    x_ = []
    y_ = []
    y_gan = []
    for i in range(window, x.shape[0]):
        tmp_x = x[i - window: i, :]
        tmp_y = y[i]
        tmp_y_gan = y[i - window: i + 1]
        x_.append(tmp_x)
        y_.append(tmp_y)
        y_gan.append(tmp_y_gan)
    x_ = torch.from_numpy(np.array(x_)).float()
    y_ = torch.from_numpy(np.array(y_)).float()
    y_gan = torch.from_numpy(np.array(y_gan)).float()
    return x_, y_, y_gan

train_x_slide, train_y_slide, train_y_gan = sliding_window(train_x, train_y, 3)
val_x_slide, val_y_slide, val_y_gan = sliding_window(val_x_scaled, val_y_scaled, 3)
test_x_slide, test_y_slide, test_y_gan = sliding_window(test_x, test_y, 3)
print(f'train_x: {train_x_slide.shape} train_y: {train_y_slide.shape} train_y_gan: {train_y_gan.shape}')
print(f'test_x: {test_x_slide.shape} test_y: {test_y_slide.shape} test_y_gan: {test_y_gan.shape}')
print(f'val_x: {val_x_slide.shape} val_y: {val_y_slide.shape} val_y_gan: {val_y_gan.shape}')


train_x: torch.Size([2623, 3, 42]) train_y: torch.Size([2623, 1]) train_y_gan: torch.Size([2623, 4, 1])
test_x: torch.Size([560, 3, 42]) test_y: torch.Size([560, 1]) test_y_gan: torch.Size([560, 4, 1])
val_x: torch.Size([560, 3, 42]) val_y: torch.Size([560, 1]) val_y_gan: torch.Size([560, 4, 1])


In [ ]:
train_x_slide, train_y_slide, train_y_gan = sliding_window(train_x, train_y, 3)
test_x_slide, test_y_slide, test_y_gan = sliding_window(test_x, test_y, 3)
print(f'train_x: {train_x_slide.shape} train_y: {train_y_slide.shape} train_y_gan: {train_y_gan.shape}')
print(f'test_x: {test_x_slide.shape} test_y: {test_y_slide.shape} test_y_gan: {test_y_gan.shape}')


train_x: torch.Size([2623, 3, 42]) train_y: torch.Size([2623, 1]) train_y_gan: torch.Size([2623, 4, 1])
test_x: torch.Size([560, 3, 42]) test_y: torch.Size([560, 1]) test_y_gan: torch.Size([560, 4, 1])


In [ ]:
class Generator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm_1 = nn.LSTM(input_size, 1024, batch_first=True)
        self.lstm_2 = nn.LSTM(1024, 512, batch_first=True)
        self.lstm_3 = nn.LSTM(512, 256, batch_first=True)
        self.linear_1 = nn.Linear(256, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        device = next(self.parameters()).device
        x = x.to(device)

        # Initialize hidden and cell states for LSTM
        h0_1 = torch.zeros(1, x.size(0), 1024).to(device)
        c0_1 = torch.zeros(1, x.size(0), 1024).to(device)
        out_1, (h1, c1) = self.lstm_1(x, (h0_1, c0_1))
        out_1 = self.dropout(out_1)

        h0_2 = torch.zeros(1, x.size(0), 512).to(device)
        c0_2 = torch.zeros(1, x.size(0), 512).to(device)
        out_2, (h2, c2) = self.lstm_2(out_1, (h0_2, c0_2))
        out_2 = self.dropout(out_2)

        h0_3 = torch.zeros(1, x.size(0), 256).to(device)
        c0_3 = torch.zeros(1, x.size(0), 256).to(device)
        out_3, (h3, c3) = self.lstm_3(out_2, (h0_3, c0_3))
        out_3 = self.dropout(out_3)

        # Fully connected layers
        out_4 = self.linear_1(out_3[:, -1, :])
        out_5 = self.linear_2(out_4)
        out_6 = self.linear_3(out_5)
        return out_6

class Discriminator(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size+1, 32, kernel_size = 5, stride = 1, padding = 'same')
        self.conv2 = nn.Conv1d(32, 64, kernel_size = 5, stride = 1, padding = 'same')
        self.batch = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size = 5, stride = 1, padding = 'same')
        self.linear1 = nn.Linear(128, 220)
        self.linear2 = nn.Linear(220, 220)
        self.linear3 = nn.Linear(220, 1)
        self.leaky = nn.LeakyReLU(0.01)
        self.relu = nn.ReLU()

    def forward(self, x):
        conv1 = self.leaky(self.conv1(x))
        conv2 = self.leaky(self.conv2(conv1))
        conv2 = self.batch(conv2)
        conv3 = self.leaky(self.conv3(conv2))
        flatten_x =  conv3.reshape(conv3.shape[0], conv3.shape[1])
        out_1 = self.leaky(self.linear1(flatten_x))
        out_2 = self.relu(self.linear2(out_1))
        out = self.linear3(out_2)
        return out

In [ ]:
def compute_gradient_penalty(critic, real_samples, fake_samples, device, lambda_term=5):
    # Create random weight term for interpolation
    batch_size = real_samples.size(0)
    epsilon_shape = [batch_size] + [1] * (real_samples.dim() - 1)
    epsilon = torch.rand(epsilon_shape, device=device)

    # Interpolate between real and fake samples.
    interpolated = epsilon * real_samples + (1 - epsilon) * fake_samples.detach()
    interpolated.requires_grad = True

    # Compute critic output on the interpolated samples.
    interpolated_scores = critic(interpolated)

    # Compute gradients of the critic outputs with respect to the interpolated samples.
    grad_outputs = torch.ones_like(interpolated_scores, device=device)
    gradients = torch.autograd.grad(
        outputs=interpolated_scores,
        inputs=interpolated,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    # Flatten gradients per sample and compute their L2 norm.
    gradients = gradients.view(batch_size, -1)
    gradient_norm = gradients.norm(2, dim=1)

    # Compute the gradient penalty as the mean squared difference from 1.
    penalty = lambda_term * ((gradient_norm - 1) ** 2).mean()
    return penalty


In [ ]:
def train_wgan(config):
  use_cuda = 1
  device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")

  trainDataloader = DataLoader(
      TensorDataset(train_x_slide, train_y_gan),
      batch_size = config["batch_size"],
      shuffle = False
  )

  modelG = Generator(42, config).to(device)
  modelD = Discriminator(3).to(device)

  optimiserG = torch.optim.Adam(
      modelG.parameters(),
      lr = config["lr"],
      betas = (0.0, 0.9),
      weight_decay = config["weight_decay"]
  )
  optimiserD = torch.optim.Adam(
      modelD.parameters(),
      lr = config["lr"],
      betas = (0.0, 0.9),
      weight_decay = config["weight_decay"]
  )

  histG = np.zeros(config["num_epochs"])
  histD = np.zeros(config["num_epochs"])
  count = 0
  for epoch in range(config["num_epochs"]):
      loss_G = []
      loss_D = []
      for (x, y) in trainDataloader:
          x = x.to(device)
          y = y.to(device)

          # Generate fake samples from the generator.
          fake_data = modelG(x)
          fake_data = torch.cat([y[:, :3, :], fake_data.reshape(-1, 1, 1)], axis=1)

          # Compute critic scores for real and fake data.
          critic_real = modelD(y)
          critic_fake = modelD(fake_data)

          # Compute the Wasserstein loss (without penalty) for the critic.
          lossD = -(torch.mean(critic_real) - torch.mean(critic_fake))

          # Compute the gradient penalty.
          gp = compute_gradient_penalty(modelD, y, fake_data, device, lambda_term=config["lambda_gp"])

          # Total critic loss includes the penalty.
          lossD_total = lossD + gp

          modelD.zero_grad()
          lossD_total.backward(retain_graph=True)
          optimiserD.step()

          # Update generator.
          output_fake = modelD(fake_data)
          lossG = -torch.mean(output_fake)
          modelG.zero_grad()
          lossG.backward()
          optimiserG.step()

          loss_D.append(lossD.item())
          loss_G.append(lossG.item())

      histG[epoch] = sum(loss_G)
      histD[epoch] = sum(loss_D)
      #print(f'[{epoch+1}/{config["num_epochs"]}] LossD: {sum(loss_D)} LossG:{sum(loss_G)}')

  modelG.eval()
  pred_y_test = modelG(test_x_slide.to(device))

  y_test_true = y_scaler.inverse_transform(test_y_slide)
  y_test_pred = y_scaler.inverse_transform(pred_y_test.cpu().detach().numpy())

  MSE = mean_squared_error(y_test_true, y_test_pred)
  RMSE = math.sqrt(MSE)

  tune.report(
      {"score": RMSE,
       "modelG" : modelG}
  )

In [ ]:

import torch.nn.functional as F

def walk_forward_prediction(config):
  modelG = config["modelG"]
  modelG.eval()

  current_window = train_x_slide[-1].unsqueeze(0).to(device)

  # Lists to store predictions and actual values.
  walk_forward_preds = []
  walk_forward_actuals = []

  # Set fine tuning parameters.
  fine_tune = True
  fine_tune_interval = config["fine_tune_interval"]
  fine_tune_epochs = config["fine_tune_epochs"]
  fine_tune_lr = config["fine_tune_lr"]
  optimizerG_ft = torch.optim.Adam(modelG.parameters(), lr=fine_tune_lr)

  for i in range(len(test_y)):
      modelG.eval()
      with torch.no_grad():
          pred = modelG(current_window)
      walk_forward_preds.append(pred.item())
      walk_forward_actuals.append(test_y[i][0])

      # --- Fine Tuning Step ---
      if fine_tune and (i % fine_tune_interval == 0):

          fine_tune_input = current_window
          actual_target = torch.tensor([[test_y[i][0]]], device=device).float()

          modelG.train()
          for epoch in range(fine_tune_epochs):
              optimizerG_ft.zero_grad()
              fine_pred = modelG(fine_tune_input)
              # Compute mean squared error loss between the prediction and the actual target.
              loss_ft = F.mse_loss(fine_pred, actual_target)
              loss_ft.backward()
              optimizerG_ft.step()
          modelG.eval()

      # --- Update the Input Window ---
      new_obs = torch.tensor(test_x[i]).float().unsqueeze(0).unsqueeze(0).to(device)
      current_window = torch.cat([current_window[:, 1:, :], new_obs], dim=1)

  modelG.eval()
  pred_y_val = modelG(val_x_slide.to(device))

  y_val_true = y_scaler.inverse_transform(val_y_slide)
  y_val_pred = y_scaler.inverse_transform(pred_y_val.cpu().detach().numpy())

  MSE = mean_squared_error(y_test_true, y_test_pred)
  RMSE = math.sqrt(MSE)

  tune.report(
      {"score": RMSE,
        "modelG" : modelG}
  )


In [ ]:
modelG_path = '/content/drive/MyDrive/Models/modelG_new.pth'
use_cuda = 0
device = torch.device("cuda" if (torch.cuda.is_available() & use_cuda) else "cpu")

modelG = Generator(42).to(device)
modelG.load_state_dict(torch.load(modelG_path, map_location=torch.device('cpu')))
modelG.eval()
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
# Define configurable hyperparameters
config = {
    "modelG" : modelG,
    "fine_tune_interval" : tune.randint(3, 20),
    "fine_tune_epochs" : tune.randint(3, 20),
    "fine_tune_lr" : tune.loguniform(1e-8, 1e-5)
}
optuna_search = OptunaSearch(
    metric="score",
    mode="min")

# Configure and run Ray Tune
tuner = tune.Tuner(
    tune.with_resources(
        walk_forward_prediction,
        resources={"cpu": 3 , "TPU": 1 if torch.cuda.is_available() else 0},
    ),
    tune_config=tune.TuneConfig(
        search_alg=optuna_search,
        num_samples=40,
        scheduler=ASHAScheduler(metric="score", mode="min"),
    ),
    run_config=tune.RunConfig(
        name="wgan_optuna_tuning",
    ),
    param_space=config
)

results = tuner.fit()

best_result = results.get_best_result(metric="score", mode="min")
print("Best config:", best_result.config)
print("Best score:", best_result.metrics["score"])

[I 2025-04-09 14:37:15,234] A new study created in memory with name: optuna


+----------------------------------------------------------+
| Configuration for experiment     wgan_optuna_tuning      |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 40                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/wgan_optuna_tuning
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-04-09_14-37-03_306744_4598/artifacts/2025-04-09_14-37-15/wgan_optuna_tuning/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-04-09 14:37:16. Total running time: 1s
Logical resource usage: 3.0/96 CPUs, 0/0 GPUs (0.0/4.0 TPU, 0.0/1.0 accelerator_type:TPU-V2)
+----------------------------------------------------------------------------------------------------------+
| Trial name                       

(walk_forward_prediction pid=14353) *** SIGSEGV received at time=1744209559 on cpu 67 ***
(walk_forward_prediction pid=14353) PC: @     0x7aa9e395b5de  (unknown)  ray::gcs::TaskInfoAccessor::AsyncAddTaskEventData()
(walk_forward_prediction pid=14353)     @     0x7aa9e4fe9520       3680  (unknown)
(walk_forward_prediction pid=14353)     @     0x7aa9e3839ce5       1392  ray::core::worker::TaskEventBufferImpl::FlushEvents()
(walk_forward_prediction pid=14353)     @     0x7aa9e37c018c       1488  ray::core::CoreWorker::Disconnect()
(walk_forward_prediction pid=14353)     @     0x7aa9e37c06bd       1152  ray::core::CoreWorker::ForceExit()
(walk_forward_prediction pid=14353)     @     0x7aa9e37c0aef       1680  ray::core::CoreWorker::HandleKillActor()
(walk_forward_prediction pid=14353)     @     0x7aa9e37b7e74        192  ray::rpc::ServerCallImpl<>::HandleRequestImpl()
(walk_forward_prediction pid=14353)     @     0x7aa9e3b8e5d8       1168  EventTracker::RecordExecution()
(walk_forward_pred


Trial status: 10 TERMINATED | 12 RUNNING | 1 PENDING
Current time: 2025-04-09 14:40:10. Total running time: 2min 54s
Logical resource usage: 39.0/96 CPUs, 0/0 GPUs (0.0/4.0 TPU, 0.0/1.0 accelerator_type:TPU-V2)
+--------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                         status         fine_tune_interval     fine_tune_epochs     fine_tune_lr     iter     total time (s)     score |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
| walk_forward_prediction_add755fd   RUNNING                         7                   17      3.11255e-07                                       |
| walk_forward_prediction_8575219a   RUNNING                         9                   18      2.74677e-06                                       |
| walk_forward_prediction_475ad657   RUNNIN

2025-04-09 14:46:47,994	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/wgan_optuna_tuning' in 203.3339s.



Trial status: 40 TERMINATED
Current time: 2025-04-09 14:46:47. Total running time: 9min 32s
Logical resource usage: 3.0/96 CPUs, 0/0 GPUs (0.0/4.0 TPU, 0.0/1.0 accelerator_type:TPU-V2)
+--------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                         status         fine_tune_interval     fine_tune_epochs     fine_tune_lr     iter     total time (s)     score |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
| walk_forward_prediction_3373359b   TERMINATED                     11                   13      9.59432e-08        1            46.2308   1.84878 |
| walk_forward_prediction_4cac60c1   TERMINATED                     15                    8      6.11517e-06        1            19.5044   1.98665 |
| walk_forward_prediction_446f97b8   TERMINATED                     1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define configurable hyperparameters
config = {
    "num_epochs": 100,
    "lr": tune.loguniform(1e-5, 1e-3),
    "batch_size": tune.choice([64, 128, 256]),
    "lstm1_units": 1024,
    "lstm2_units": 512,
    "lstm3_units": 256,
    "lambda_term": tune.uniform(5, 15),
    "dropout_rate": tune.uniform(0.1, 0.3),
    "weight_decay": tune.loguniform(1e-4, 1e-2),
    "lambda_gp": tune.uniform(1, 10)
}
optuna_search = OptunaSearch(
    metric="score",
    mode="min")

# Configure and run Ray Tune
tuner = tune.Tuner(
    tune.with_resources(
        train_wgan,  # Your training function
        resources={"cpu": 3 , "TPU": 1 if torch.cuda.is_available() else 0},  # Resource specification
    ),
    tune_config=tune.TuneConfig(
        search_alg=optuna_search,
        num_samples=32,  # Number of trials
        scheduler=ASHAScheduler(metric="score", mode="min"),
    ),
    run_config=tune.RunConfig(
        name="wgan_optuna_tuning",
    ),
    param_space=config
)

results = tuner.fit()

best_result = results.get_best_result(metric="score", mode="min")
print("Best config:", best_result.config)

[I 2025-03-19 19:29:43,706] A new study created in memory with name: optuna


+----------------------------------------------------------+
| Configuration for experiment     wgan_optuna_tuning      |
+----------------------------------------------------------+
| Search algorithm                 SearchGenerator         |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 30                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/wgan_optuna_tuning
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-19_18-58-37_873415_5705/artifacts/2025-03-19_19-29-43/wgan_optuna_tuning/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-03-19 19:29:43. Total running time: 0s
Logical resource usage: 4.0/96 CPUs, 0/0 GPUs (0.0/4.0 TPU, 0.0/1.0 accelerator_type:TPU-V2)
+-----------------------------------------------------------------------------------------------------------------------------+
| Trial name    

2025-03-19 20:08:27,512	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/wgan_optuna_tuning' in 24.2149s.



Trial status: 30 TERMINATED
Current time: 2025-03-19 20:08:27. Total running time: 38min 43s
Logical resource usage: 4.0/96 CPUs, 0/0 GPUs (0.0/1.0 accelerator_type:TPU-V2, 0.0/4.0 TPU)
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name            status                lr     batch_size     lambda_term     dropout_rate     weight_decay     lambda_gp     iter     total time (s)       score |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_wgan_db8ac253   TERMINATED   0.000495601             64        13.8053          0.267784      0.000397004       9.77822        1           1764.79    123.566   |
| train_wgan_5a5ce878   TERMINATED   0.000161198            256         9.19586         0.223712      0.00291426        8.22799      

In [ ]:
best_result = results.get_best_result(metric="score", mode="min")
best_result.metrics
print("Best config:", best_result.config)
print(best_result.metrics)

Best config: {'num_epochs': 100, 'lr': 2.3027243533728277e-05, 'batch_size': 128, 'lstm1_units': 1024, 'lstm2_units': 512, 'lstm3_units': 256, 'lambda_term': 9.583566888471712, 'dropout_rate': 0.12968382925633937, 'weight_decay': 0.00020253997342129375, 'lambda_gp': 5.303186839680299}
{'score': 6.267411540166204, 'modelG': Generator(
  (lstm_1): LSTM(42, 1024, batch_first=True)
  (lstm_2): LSTM(1024, 512, batch_first=True)
  (lstm_3): LSTM(512, 256, batch_first=True)
  (linear_1): Linear(in_features=256, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=64, bias=True)
  (linear_3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.12968382925633937, inplace=False)
), 'timestamp': 1742414115, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '6c5fec5b', 'date': '2025-03-19_19-55-15', 'time_this_iter_s': 1428.5652940273285, 'time_total_s': 1428.5652940273285, 'pid': 493504, 'hostname': '40edbf86f019', 'n